# Import

## Packages

In [1]:
import numpy as np
import pandas as pd

import re

## Data

In [17]:
# columns definition

# col = 'district(鄉鎮市區)、rps01(交易標的)、rps02(土地區段位置建物區段門牌)、rps03(土地移轉總面積平方公尺)、rps04(都市土地使用分區)、rps05(非都市土地使用分區)、rps06(非都市土地使用編定)、rps07(交易年月日)、rps08(交易筆棟數)、rps09(移轉層次)、rps10(總樓層數)、rps11(建物型態)、rps12(主要用途)、rps13(主要建材)、rps14(建築完成年月)、rps15(建物移轉總面積平方公尺)、rps16(建物現況格局-房)、rps17(建物現況格局-廳)、rps18(建物現況格局-衛)、rps19(建物現況格局-隔間)、rps20(有無管理組織)、rps21(總價元)、rps22(單價元平方公尺)、rps23(車位類別)、rps24(車位移轉總面積平方公尺)、rps25(車位總價元)、rps26(備註)、rps27(編號)、rps28(主建物面積)、rps29(附屬建物面積)、rps30(陽台面積)、rps31(電梯)'

# col_name = []
# col_desc = []
# col = col.split('、')

# for i in range(len(col)):
#     content = col[i].split('(')    
#     col_name.append(content[0])
#     col_desc.append(content[1][:-1])

# data_desc = pd.DataFrame()
# data_desc['col_name'] = col_name
# data_desc['col_desc'] = col_desc

# data_desc.to_csv('data/data_description.csv', index=False)

In [17]:
df_21Q1_1 = pd.read_csv('data/2021Q1/A_lvr_land_A.csv', error_bad_lines=False)
df_21Q1_2 = pd.read_csv('data/2021Q1/A_lvr_land_B.csv', error_bad_lines=False)
df_21Q1_3 = pd.read_csv('data/2021Q1/A_lvr_land_C.csv', error_bad_lines=False)

b'Skipping line 80: expected 29 fields, saw 31\nSkipping line 82: expected 29 fields, saw 30\nSkipping line 241: expected 29 fields, saw 30\nSkipping line 567: expected 29 fields, saw 32\nSkipping line 774: expected 29 fields, saw 30\nSkipping line 901: expected 29 fields, saw 30\nSkipping line 920: expected 29 fields, saw 32\nSkipping line 1020: expected 29 fields, saw 30\nSkipping line 1021: expected 29 fields, saw 30\nSkipping line 1070: expected 29 fields, saw 30\nSkipping line 1086: expected 29 fields, saw 30\nSkipping line 1255: expected 29 fields, saw 31\nSkipping line 1256: expected 29 fields, saw 31\n'


In [18]:
print(df_21Q1_1.shape)
print(df_21Q1_2.shape)
print(df_21Q1_3.shape)

(6520, 33)
(205, 28)
(1327, 29)


# Data process

## columns names setting

In [323]:
col = pd.read_csv('data/temp_tables/column_names.csv')
col_names = col['col_name'].tolist()

In [339]:
df = pd.read_csv('data/2021Q1/A_lvr_land_A.csv')

# There're two rows about header
df = df.iloc[1:, :]
df.columns = col_names

# Need to reset index because exclude one row 
df = df.reset_index(drop=True)

In [53]:
df.dtypes

district                        object
transaction_sign                object
address                         object
land_area                       object
use_zone                        object
non_metropolis_use_district     object
non_metropolis_use              object
yyymmdd                         object
transaction_number              object
floor_ch                        object
total_floor_ch                  object
building_state                  object
main_use                        object
building_materials              object
building_yyymmdd                object
building_area                   object
room                            object
hall                            object
health                          object
compartment                     object
manage                          object
total_price                     object
unit_price                      object
berth_type                      object
berth_area                      object
berth_price              

## District

In [343]:
district = pd.read_csv('data/temp_tables/district.csv')

In [344]:
df = df.merge(district, how='left', on='district')

## use zone

In [345]:
df.groupby('use_zone').size().rename('count').reset_index().sort_values(by='count', ascending=False).head()

,use_zone,count
139,都市：其他:第三種住宅區。,2822
147,都市：其他:第三種商業區。,376
242,都市：其他:第肆種商業區(依都市計畫說明書圖規定辦理，始得作第肆種商業區使用)(原屬第參種商...,258
155,都市：其他:第二種住宅區。,252
220,都市：其他:第四種住宅區。,198


In [346]:
df['residential_zone'] = np.where((df['use_zone'].str.find('住宅區')>=0)|(df['use_zone'].str.find('住宅用地')>=0), 1, 0)
df['business_zone'] = np.where(df['use_zone'].str.find('商業區')>=0, 1, 0)

## time

In [347]:
df['buy_yyyy'] = df['yyymmdd'].str[:-4].astype(int)+1911
df['buy_mm'] = df['yyymmdd'].str[-4:-2]

There're missing values in build time

In [348]:
df['build_yyyy'] = df['building_yyymmdd'].str[:-4]
df['build_yyyy'] = pd.to_numeric(df['build_yyyy'], errors='coerce')
df['build_yyyy'] = df['build_yyyy']+1911

df['build_mm'] = df['building_yyymmdd'].str[-4:-2]
df['build_mm'] = pd.to_numeric(df['build_mm'], errors='coerce')

In [349]:
df[['building_yyymmdd', 'build_yyyy', 'build_mm']]

,building_yyymmdd,build_yyyy,build_mm
0,0850715,1996.0,7.0
1,0760413,1987.0,4.0
2,0570614,1968.0,6.0
3,NaN,NaN,NaN
4,1090825,2020.0,8.0
...,...,...,...
6514,0870213,1998.0,2.0
6515,NaN,NaN,NaN
6516,NaN,NaN,NaN
6517,NaN,NaN,NaN


## transaction number

In [350]:
df['land'] = 0
df['building'] = 0
df['berth'] = 0

for i in range(0, len(df)):
    df['land'][i] = re.split(r'(\d+)', df['transaction_number'][i])[1]
    df['building'][i] = re.split(r'(\d+)', df['transaction_number'][i])[3]
    df['berth'][i] = re.split(r'(\d+)', df['transaction_number'][i])[5]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## floor

convert chinese to Arabic digit

### total floor

In [351]:
df['total_floor_ch'] = df['total_floor_ch'].str[:-1]
df['total_floor'] = 0

for i in range(0, len(df)):
    num = re.split(r'(\d+)', str(df['total_floor_ch'][i]).replace('十', '10'))
    num = list(map(lambda e: e.replace('一', '1').replace('二', '2').replace('三', '3').replace('四', '4').replace('五', '5')\
                  .replace('六', '6').replace('七', '7').replace('八', '8').replace('九', '9'), num))
    
    if (len(num) == 1) & (num[0] != 'nan'):
        df['total_floor'][i] = num[0]
    elif len(num) == 3:
        if num[0] == '':
            if num[2] == '':
                df['total_floor'][i] = 10
            else:
                df['total_floor'][i] = int(num[1])+int(num[2])
        else:
            if num[2] == '':
                df['total_floor'][i] = int(num[0])*int(num[1])
            else:
                df['total_floor'][i] = int(num[0])*int(num[1])+int(num[2])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/si

### shift floor

In [352]:
# fill na to prevent error
df['floor_ch'] = df['floor_ch'].fillna('')

# extract the first element
df['floor_ch'] = df['floor_ch'].apply(lambda x: x.split('，')[0][:-1])

In [353]:
df['floor_ch'] = np.where((df['floor_ch'].str.contains('地下'))|(df['floor_ch'].str.contains('停車'))|
                          (df['floor_ch'].str.contains('避難'))|(df['floor_ch'].str.contains('樓梯')),
                          '', df['floor_ch'])

In [354]:
df['floor'] = 0

for i in range(0, len(df)):
    num = re.split(r'(\d+)', str(df['floor_ch'][i]).replace('十', '10'))
    num = list(map(lambda e: e.replace('一', '1').replace('二', '2').replace('三', '3').replace('四', '4').replace('五', '5')\
                  .replace('六', '6').replace('七', '7').replace('八', '8').replace('九', '9'), num))
    
    if (len(num) == 1) & (num[0] != 'nan'):
        df['floor'][i] = num[0]
    elif len(num) == 3:
        if num[0] == '':
            if num[2] == '':
                df['floor'][i] = 10
            else:
                df['floor'][i] = int(num[1])+int(num[2])
        else:
            if num[2] == '':
                df['floor'][i] = int(num[0])*int(num[1])
            else:
                df['floor'][i] = int(num[0])*int(num[1])+int(num[2])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/pyt

## value recode

### transaction sign

In [355]:
df['transaction_sign'] = np.where(df['transaction_sign']=='房地(土地+建物)+車位', 1, 
                                 np.where(df['transaction_sign']=='房地(土地+建物)', 2,
                                          np.where(df['transaction_sign']=='建物', 3,
                                                   np.where(df['transaction_sign']=='車位', 4, 5
                                                   ))))

### building state

1: 住宅大樓(11層含以上有電梯)
2: 華廈(10層含以下有電梯)
3: 公寓(5樓含以下無電梯)
4: 透天厝
5: 其他

In [356]:
df.groupby('building_state').size().rename('count').reset_index().sort_values(by='count', ascending=False)

,building_state,count
0,住宅大樓(11層含以上有電梯),2544
4,華廈(10層含以下有電梯),1609
1,公寓(5樓含以下無電梯),1509
2,其他,680
6,透天厝,161
3,廠辦,15
5,辦公商業大樓,1


In [357]:
df['building_state'] = np.where(df['building_state']=='住宅大樓(11層含以上有電梯)', 1, 
                               np.where(df['building_state']=='華廈(10層含以下有電梯)', 2,
                                       np.where(df['building_state']=='公寓(5樓含以下無電梯)', 3,
                                                np.where(df['building_state']=='透天厝', 4, 5))))

### use type

1: 住家
2: 商業
3: 停車場
4: 無法判斷

In [121]:
# use = df.groupby('main_use').size().rename('count').reset_index().sort_values(by='count', ascending=False)
# use.to_csv('data/main_use.csv', index=False)

In [358]:
use = pd.read_csv('data/temp_tables/main_use.csv')

In [359]:
df = df.merge(use, how='left', on='main_use')

In [360]:
df[['main_use', 'use_type']]

,main_use,use_type
0,住家用,1.0
1,住家用,1.0
2,見其他登記事項,4.0
3,住家用,1.0
4,見其他登記事項,4.0
...,...,...
6514,住家用,1.0
6515,住家用,1.0
6516,住家用,1.0
6517,NaN,NaN


### building_materials

In [361]:
df['building_materials'] = np.where(df['building_materials']=='鋼筋混凝土造', 1,
                                   np.where((df['building_materials']=='加強磚造')|(df['building_materials']=='磚造'), 2, 3))

### compartment

In [362]:
df['compartment'] = np.where(df['compartment'] == '有', 1, 0)

### management

In [363]:
df['management'] = np.where(df['management'] == '有', 1, 0)

### elevator

In [364]:
df['elevator'] = np.where(df['elevator'] == '有', 1, 0)

### berth type

In [232]:
# berth = df.groupby('berth_category').size().rename('count').reset_index().sort_values(by='count', ascending=False)
# berth.to_csv('data/berth_type.csv', index=False)

In [365]:
berth = pd.read_csv('data/temp_tables/berth_type.csv')

In [366]:
df = df.merge(berth, how='left', on='berth_category')

## address

### road section

In [367]:
df['road_section'] = 0

for i in range(0, len(df)):
    road = re.split(r'(\d+)', df['address'][i])
    df['road_section'][i] = road[0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [368]:
df[['address', 'road_section']].head()

,address,road_section
0,興隆路二段217巷1~30號,興隆路二段
1,和平東路四段65巷77弄1~30號,和平東路四段
2,羅斯福路六段26巷1~30號,羅斯福路六段
3,興隆路四段145巷31~60號,興隆路四段
4,興隆路三段192巷2弄1~30號,興隆路三段


In [389]:
df['address'][0]

'興隆路二段217巷1~30號'

### coordinate

In [404]:
df['address_exact'] = ''

for i in range(0, len(df)):
    address_s = ''
    s = re.split(r'(\d+)', df['address'][i])
    
    if len(s) >= 5:  
        for j in range(len(s)-4):
            address_s += s[j]
        address_s += str(int(s[-2])-15)
        address_s += s[-1]

    df['address_exact'][i] = address_s
    
df['address_exact'] = '台北市'+df['district']+df['address_exact']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [405]:
df[['district', 'address', 'address_exact']]

,district,address,address_exact
0,文山區,興隆路二段217巷1~30號,台北市文山區興隆路二段217巷15號
1,文山區,和平東路四段65巷77弄1~30號,台北市文山區和平東路四段65巷77弄15號
2,文山區,羅斯福路六段26巷1~30號,台北市文山區羅斯福路六段26巷15號
3,文山區,興隆路四段145巷31~60號,台北市文山區興隆路四段145巷45號
4,文山區,興隆路三段192巷2弄1~30號,台北市文山區興隆路三段192巷2弄15號
...,...,...,...
6514,文山區,萬美街二段61~90號,台北市文山區萬美街二段75號
6515,文山區,興隆路一段271巷4弄1~30號,台北市文山區興隆路一段271巷4弄15號
6516,文山區,景華街82巷1~30號,台北市文山區景華街82巷15號
6517,中正區,延平南路61~90號,台北市中正區延平南路75號
